In [3]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [6]:
os.listdir('artifacts/2020-06-21T212459Z/')
lastdir = 'artifacts/2020-06-21T212459Z/'

In [7]:
bundle = joblib.load(f'{lastdir}/bundle_0.joblib')

In [17]:
np.sum(bundle['metrics']['confusion_matrix'], axis=0)


array([421708,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0])

Yea taking quick look at that model using the functional xgb api instead of the sklearn API, indeed per above, zero learning is happening there. SOmething needs some major tweaking there.

#### Continue unravel  the functional xgb API vs sklearn xgb API rseult..
* I want to build on [past notebook](https://github.com/namoopsoo/learn-citibike/blob/2020-revisit/notes/2020-06-21.md) , where I 

In [18]:

dataset_loc = '/opt/program/artifacts/2020-06-26T142154Z/train.csv'
#dtrain = xgb.DMatrix(f'{outpath}#dtrain.cache')

dmatrix = xgb.DMatrix(
    f'{dataset_loc}?format=csv&label_column=0&delimiter=,')


[19:07:56] 843416x85 matrix with 71690360 entries loaded from /opt/program/artifacts/2020-06-26T142154Z/train.csv?format=csv&label_column=0&delimiter=,


In [19]:
fu.get_my_memory()

{'pmem': '34.9', 'rss': '0.681 GiB'}

In [20]:
#dmatrix = xgb.DMatrix(f'{outpath}#dtrain.cache')
dmatrix = xgb.DMatrix(
    f'{dataset_loc}?format=csv&label_column=0&delimiter=,#dtrain.cache')


[19:09:28] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/2020-06-26T142154Z/train.csv?format=csv&label_column=0&delimiter=,#dtrain.cache


In [28]:
# hmm this is weird?
# dmatrix.get_label() # array([1.], dtype=float32)  ...

# what the heck? 
# dmatrix.num_col(), dmatrix.num_row()
# => (78, 1)

# When loading w/o cache I saw the message was looking good..
# "843416x85 matrix with 71690360 entries loaded" 
# And 71690360 => 843416*85 makes sense 

# But loading w/ the "#dtrain.cache" I see strangely 
# "1x78 matrix with 16000 entries loaded"
# Huh?

# Quite possible that this caching is only allowed w/ the "libsvm" format
# and not the csv format.
# At least that is slightly mentioned , here, 
# , https://github.com/dmlc/xgboost/blob/master/demo/guide-python/external_memory.py


#### try w/ libsvm...
Not doing cross validation yet, just want to see if this even works w/o crashing yet. And I want to see if the log is still looking like no learning is happening.

In [35]:
#dmatrix = xgb.DMatrix(f'{outpath}#dtrain.cache')
dataset_loc = '/opt/program/artifacts/2020-06-26T142154Z/train.libsvm'
dtrain = xgb.DMatrix(
    f'{dataset_loc}?format=libsvm#dtrain.cache')



[22:59:23] 1x78 matrix with 16000 entries loaded from /opt/program/artifacts/2020-06-26T142154Z/train.libsvm?format=libsvm#dtrain.cache


In [37]:
params = {'max_depth':3, 
          'learning_rate': .1, # 'eta':0.1   # alias
          'objective':'multi:softprob',   # mlogloss? 
          'num_class':  54 ,
          'base_score':0.5, 
          'booster':'gbtree', 
          'colsample_bylevel':1,
          'colsample_bynode':1, 
          'colsample_bytree':1, 
          'gamma':0,
          'max_delta_step':0, 
          'min_child_weight':1, #'missing':nan, 
          'random_state':0,
          'reg_alpha':0, 
          'reg_lambda':1,
          'scale_pos_weight':1, 
          'seed': 42,
          #'silent':None, 
          'subsample':1, 
          'verbosity': 2
          
          # from sklearn...
          # 'n_estimators':100, 'n_jobs':1,
         }

watchlist = [(dtrain, 'train')]
num_round = 100

xgb_model = xgb.train(params, dtrain, num_round, watchlist)



[23:00:16] WARNING: src/learner.cc:658: Tree method is automatically set to 'approx' since external-memory data matrix is used.
[23:00:16] SparsePageSource: Finished writing to dtrain.cache
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max

[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[1]	train-merror:1
[23:00:17] INFO: src/tree/updater_prune.cc:74

[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:17] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:18] INFO: src/tree/updater_prune.cc:74: tree pruning end,

...

[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:34] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[95]	train-merror:1
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:7

[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end,

[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:35] INFO: src/tree/updater_prune.cc:74: tree pruning end,

#### some of the log 
* the log from the above was very verbose, but nothing changing at all, so just showing some...

```
[23:00:16] WARNING: src/learner.cc:658: Tree method is automatically set to 'approx' since external-memory data matrix is used.
[23:00:16] SparsePageSource: Finished writing to dtrain.cache
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:16] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
...
...
[23:00:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[23:00:36] INFO: src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[99]	train-merror:1
```
* The "Tree method is automatically set to 'approx' since external-memory data matrix is used." message was pretty interesting. Wonder if that's impacting the performance?

### 2020-06-28

#### ok killing kernel and try for real..

In [1]:
!top

=top - 17:42:13 up 7 days,  8:04,  0 users,  load average: 0.25, 0.13, 0.10
Tasks:   5 total,   1 running,   4 sleeping,   0 stopped,   0 zombie
%Cpu(s):  6.0 us,  0.5 sy,  0.0 ni, 93.3 id,  0.1 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem :  2046844 total,  1767972 free,   145292 used,   133580 buff/cache
KiB Swap:  1048572 total,   820096 free,   228476 used.  1748448 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
 1053 root      20   0  855260  31420   6208 S   6.7  1.5   0:43.95 jupyter-no+ 
 1519 root      20   0  561768  52496  13780 S   6.7  2.6   0:00.17 python      
    1 root      20   0   18236      0      0 S   0.0  0.0   0:00.22 bash        
 1533 root      20   0    4500    736    664 S   0.0  0.0   0:00.51 sh          
 1534 root      20   0   36852   3252   2840 R   0.0  0.2   0:00.00 top         
>
